In [1]:
import json
from selenium import webdriver
from bs4 import BeautifulSoup

### Get the website

In [2]:
# Create an instance of ChromeOptions
options = webdriver.ChromeOptions()
# Run headless mode so browser does not continually open with every iteration
options.add_argument("headless")
# Instatiate chrome driver and pass in the file path to chromedriver
driver = webdriver.Chrome('/Users/kevinmacmat/Desktop/capstone_project/chromedriver', options=options)

In [3]:
dbn_list = ['29Q034', '01M034', '03M165', '02M347', '03M149']

In [4]:
# Get website
driver.get('https://insideschools.org/school/01M034')

### Switch to the iframe containing the script element with the comments sections

In [5]:
# driver.switch_to.frame(driver.find_element_by_tag_name("iframe"))
driver.switch_to.frame(1)
text = driver.page_source
driver.switch_to.default_content()

### Import BeautifulSoup to pull the script tag

In [6]:
# Parse and process the source with BeautifulSoup module by creating an BS object based on src
soup = BeautifulSoup(text, 'lxml')

### Save the disqus thread to the variable: "thread"

In [7]:
# Access the soup and use the find method. Pass in the script element
thread = soup.find("script", {"id": "disqus-threadData"})

### Access the json

In [8]:
site_json=json.loads(thread.text)

### Loop through and save comments as a list.

In [9]:
comments_list = []

for x in site_json['response']['posts']:
    comments_list.append(x['raw_message'])

In [10]:
len(comments_list)

27

In [11]:
comments_list[16]

'Jenny file a complaint they think they could do what they want Mr.Jacobs is bad none of them do there job right'